# **ETL GOOGLE MAPS**

In [1]:
!pip install nltk

StatementMeta(, 652a407c-4baa-4bc2-bae0-2be59ae1e377, 3, Finished, Available)

**FUNCIONES**

In [3]:
def info_data_df(df):
    '''
    Esta función proporciona información detallada sobre el tipo de dato y los valores nulos presentes en un marco de datos.
    '''

    info_dict = {"Columnas": [], "Tipo_Dato": [], "No_miss_Qty": [], "%Missing": [], "Missing_Qty": []}
    

    for column in df.columns:
        
        info_dict["Columnas"].append(column)
        info_dict["Tipo_Dato"].append(df[column].apply(type).unique())
        info_dict["No_miss_Qty"].append(df[column].count())
        info_dict["%Missing"].append(round(df[column].isnull().sum() * 100 / len(df), 2))
        info_dict['Missing_Qty'].append(df[column].isnull().sum())

    df_info = pd.DataFrame(info_dict)
    
    print("\nTotal filas: ", len(df))
    print("\nTotal filas nulas: ", df.isna().all(axis=1).sum())
    print("\nTotal filas duplicadas: ", df.duplicated().sum())
    
    
    return df_info


StatementMeta(, 652a407c-4baa-4bc2-bae0-2be59ae1e377, 5, Finished, Available)

**IMPORTAMOS LIBRERIAS**

In [2]:
# Librerias a utilizar

import os
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pickle
import ast
from joblib import Parallel, delayed
import nltk


StatementMeta(, 652a407c-4baa-4bc2-bae0-2be59ae1e377, 4, Finished, Available)

**LEEMOS DATA NECESARIA DE GOOGLE MAPS**

In [4]:
# Load data into pandas DataFrame from "/lakehouse/default/" + "Files/DATA FINAL/GOOGLE MAPS/estados.csv"
df_google_estados = pd.read_csv("/lakehouse/default/" + "Files/DATA FINAL/GOOGLE MAPS/estados.csv")



StatementMeta(, 652a407c-4baa-4bc2-bae0-2be59ae1e377, 6, Finished, Available)

In [5]:
df_google_estados

StatementMeta(, 652a407c-4baa-4bc2-bae0-2be59ae1e377, 7, Finished, Available)

,Unnamed: 0,user_id,name,time,rating,text,pics,resp,gmap_id,estado
0,0,1.140438e+20,Kanisha Mixon,1597168272670,5,Very Personable staff! Beautiful and clean env...,NaN,NaN,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama
1,1,1.160090e+20,Brandie Hodges,1609899039594,5,Best clothing intown,NaN,NaN,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama
2,2,1.062399e+20,Sharon King,1547235290843,4,NaN,NaN,NaN,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama
3,3,1.049701e+20,Veronica Pierce,1517709403534,5,NaN,NaN,NaN,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama
4,4,1.105875e+20,Whitney Waldon Collier,1535245718492,5,NaN,NaN,NaN,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama
...,...,...,...,...,...,...,...,...,...,...
81396354,81396354,1.078991e+20,Carlene Calabaza,1614741323428,5,NaN,NaN,NaN,0x53349429523d615b:0x2a66a9272032e8b4,Wyoming
81396355,81396355,1.173878e+20,Gizmo,1561931911835,5,NaN,NaN,NaN,0x53349429523d615b:0x2a66a9272032e8b4,Wyoming
81396356,81396356,1.078871e+20,Jan Soucek,1501029366785,5,NaN,NaN,NaN,0x53349429523d615b:0x2a66a9272032e8b4,Wyoming
81396357,81396357,1.047652e+20,Kyle Leatherow,1565466860108,3,NaN,NaN,NaN,0x53349429523d615b:0x2a66a9272032e8b4,Wyoming


##### **TRATATREMOS NULOS Y DUPLICADOS**

###### **PARA LA DATA DE ESTADOS:**

In [7]:
info_data_df(df_google_estados)

StatementMeta(, 652a407c-4baa-4bc2-bae0-2be59ae1e377, 9, Finished, Available)


Total filas:  81396359

Total filas nulas:  0

Total filas duplicadas:  0


,Columnas,Tipo_Dato,No_miss_Qty,%Missing,Missing_Qty
0,Unnamed: 0,[<class 'int'>],81396359,0.00,0
1,user_id,[<class 'float'>],81396359,0.00,0
2,name,"[<class 'str'>, <class 'float'>]",81395909,0.00,450
3,time,[<class 'int'>],81396359,0.00,0
4,rating,[<class 'int'>],81396359,0.00,0
5,text,"[<class 'str'>, <class 'float'>]",45541004,44.05,35855355
6,pics,"[<class 'float'>, <class 'str'>]",2226417,97.26,79169942
7,resp,"[<class 'float'>, <class 'str'>]",9852916,87.90,71543443
8,gmap_id,[<class 'str'>],81396359,0.00,0
9,estado,[<class 'str'>],81396359,0.00,0


In [8]:
# Primero, convierte los valores de tiempo Unix en objetos de fecha y hora
df_google_estados['time'] = pd.to_datetime(df_google_estados['time'], unit='ms')

StatementMeta(, 652a407c-4baa-4bc2-bae0-2be59ae1e377, 10, Finished, Available)

In [9]:
df_google_estados

StatementMeta(, 652a407c-4baa-4bc2-bae0-2be59ae1e377, 11, Finished, Available)

,Unnamed: 0,user_id,name,time,rating,text,pics,resp,gmap_id,estado
0,0,1.140438e+20,Kanisha Mixon,2020-08-11 17:51:12.670,5,Very Personable staff! Beautiful and clean env...,NaN,NaN,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama
1,1,1.160090e+20,Brandie Hodges,2021-01-06 02:10:39.594,5,Best clothing intown,NaN,NaN,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama
2,2,1.062399e+20,Sharon King,2019-01-11 19:34:50.843,4,NaN,NaN,NaN,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama
3,3,1.049701e+20,Veronica Pierce,2018-02-04 01:56:43.534,5,NaN,NaN,NaN,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama
4,4,1.105875e+20,Whitney Waldon Collier,2018-08-26 01:08:38.492,5,NaN,NaN,NaN,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama
...,...,...,...,...,...,...,...,...,...,...
81396354,81396354,1.078991e+20,Carlene Calabaza,2021-03-03 03:15:23.428,5,NaN,NaN,NaN,0x53349429523d615b:0x2a66a9272032e8b4,Wyoming
81396355,81396355,1.173878e+20,Gizmo,2019-06-30 21:58:31.835,5,NaN,NaN,NaN,0x53349429523d615b:0x2a66a9272032e8b4,Wyoming
81396356,81396356,1.078871e+20,Jan Soucek,2017-07-26 00:36:06.785,5,NaN,NaN,NaN,0x53349429523d615b:0x2a66a9272032e8b4,Wyoming
81396357,81396357,1.047652e+20,Kyle Leatherow,2019-08-10 19:54:20.108,3,NaN,NaN,NaN,0x53349429523d615b:0x2a66a9272032e8b4,Wyoming


In [11]:
# Convertir la columna 'time' al tipo de datos de cadena
df_google_estados['time'] = df_google_estados['time'].astype(str)
# Dividir la columna 'time' en dos columnas separadas: 'fecha' y 'hora'
df_google_estados[['fecha', 'hora']] = df_google_estados['time'].str.split(' ', expand=True)

StatementMeta(, 652a407c-4baa-4bc2-bae0-2be59ae1e377, 13, Finished, Available)

In [12]:
df_google_estados

StatementMeta(, 652a407c-4baa-4bc2-bae0-2be59ae1e377, 14, Finished, Available)

,Unnamed: 0,user_id,name,time,rating,text,pics,resp,gmap_id,estado,fecha,hora
0,0,1.140438e+20,Kanisha Mixon,2020-08-11 17:51:12.670,5,Very Personable staff! Beautiful and clean env...,NaN,NaN,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama,2020-08-11,17:51:12.670
1,1,1.160090e+20,Brandie Hodges,2021-01-06 02:10:39.594,5,Best clothing intown,NaN,NaN,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama,2021-01-06,02:10:39.594
2,2,1.062399e+20,Sharon King,2019-01-11 19:34:50.843,4,NaN,NaN,NaN,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama,2019-01-11,19:34:50.843
3,3,1.049701e+20,Veronica Pierce,2018-02-04 01:56:43.534,5,NaN,NaN,NaN,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama,2018-02-04,01:56:43.534
4,4,1.105875e+20,Whitney Waldon Collier,2018-08-26 01:08:38.492,5,NaN,NaN,NaN,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama,2018-08-26,01:08:38.492
...,...,...,...,...,...,...,...,...,...,...,...,...
81396354,81396354,1.078991e+20,Carlene Calabaza,2021-03-03 03:15:23.428,5,NaN,NaN,NaN,0x53349429523d615b:0x2a66a9272032e8b4,Wyoming,2021-03-03,03:15:23.428
81396355,81396355,1.173878e+20,Gizmo,2019-06-30 21:58:31.835,5,NaN,NaN,NaN,0x53349429523d615b:0x2a66a9272032e8b4,Wyoming,2019-06-30,21:58:31.835
81396356,81396356,1.078871e+20,Jan Soucek,2017-07-26 00:36:06.785,5,NaN,NaN,NaN,0x53349429523d615b:0x2a66a9272032e8b4,Wyoming,2017-07-26,00:36:06.785
81396357,81396357,1.047652e+20,Kyle Leatherow,2019-08-10 19:54:20.108,3,NaN,NaN,NaN,0x53349429523d615b:0x2a66a9272032e8b4,Wyoming,2019-08-10,19:54:20.108


In [13]:
#Borramos columnas que desestimaremos del estudio:
drop_col=['resp', 'pics','time','Unnamed: 0','hora']
df_google_estados.drop(columns=drop_col, inplace=True)
#Borramos filas duplicadas:
df_google_estados.drop_duplicates(inplace=True)

StatementMeta(, 652a407c-4baa-4bc2-bae0-2be59ae1e377, 15, Finished, Available)

In [14]:
#Analizamos la columna "estado" para luego reducir la data con respecto a los estados de mayor impacto en el sector hotelero.
estados_unicos = df_google_estados['estado'].unique()
lista_estados_unicos = list(estados_unicos)
print(lista_estados_unicos)

StatementMeta(, 652a407c-4baa-4bc2-bae0-2be59ae1e377, 16, Finished, Available)

['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']


In [15]:
#Lista de estados seleccionados
estados_seleccionados= ['California','Florida','Nevada','New York','Massachusetts','Mississippi','Oregon','illinois','Louisiana','Alabama','Texas','Washington','Georgia','North Carolina','South Carolina','Virginia','Maryland','Delaware','New jersey']
#filtramos nuestro df
df_google_estados= df_google_estados[df_google_estados['estado'].isin(estados_seleccionados)]

StatementMeta(, 652a407c-4baa-4bc2-bae0-2be59ae1e377, 17, Finished, Available)

In [16]:
#Visualizamos la info de nuestro df de estados
info_data_df(df_google_estados)

StatementMeta(, 652a407c-4baa-4bc2-bae0-2be59ae1e377, 18, Finished, Available)


Total filas:  27619402

Total filas nulas:  0

Total filas duplicadas:  0


,Columnas,Tipo_Dato,No_miss_Qty,%Missing,Missing_Qty
0,user_id,[<class 'float'>],27619402,0.00,0
1,name,"[<class 'str'>, <class 'float'>]",27619248,0.00,154
2,rating,[<class 'int'>],27619402,0.00,0
3,text,"[<class 'str'>, <class 'float'>]",15428446,44.14,12190956
4,gmap_id,[<class 'str'>],27619402,0.00,0
5,estado,[<class 'str'>],27619402,0.00,0
6,fecha,[<class 'str'>],27619402,0.00,0


In [17]:
# Reemplazar NaN por 'S/D' en la columna 'text'
df_google_estados['text'].fillna('S/D', inplace=True)

StatementMeta(, 652a407c-4baa-4bc2-bae0-2be59ae1e377, 19, Finished, Available)

/tmp/ipykernel_5644/463882217.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_google_estados['text'].fillna('S/D', inplace=True)


In [18]:
df_google_estados

StatementMeta(, 652a407c-4baa-4bc2-bae0-2be59ae1e377, 20, Finished, Available)

,user_id,name,rating,text,gmap_id,estado,fecha
0,1.140438e+20,Kanisha Mixon,5,Very Personable staff! Beautiful and clean env...,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama,2020-08-11
1,1.160090e+20,Brandie Hodges,5,Best clothing intown,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama,2021-01-06
2,1.062399e+20,Sharon King,4,S/D,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama,2019-01-11
3,1.049701e+20,Veronica Pierce,5,S/D,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama,2018-02-04
4,1.105875e+20,Whitney Waldon Collier,5,S/D,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama,2018-08-26
...,...,...,...,...,...,...,...
78201731,1.108662e+20,Dualle Abdi,5,S/D,0x54906c4bb4541b15:0x167db6becb362e15,Washington,2020-06-23
78201732,1.161065e+20,YURII HULYI,5,S/D,0x54906c4bb4541b15:0x167db6becb362e15,Washington,2020-09-12
78201733,1.067696e+20,Imran Aslam,5,S/D,0x54906c4bb4541b15:0x167db6becb362e15,Washington,2018-06-16
78201734,1.011000e+20,ahmed alsalihi,5,S/D,0x54906c4bb4541b15:0x167db6becb362e15,Washington,2019-07-17


##### **Aplicamos el analisis de sentimientos a la columna 'text', optimizando el codigo con el uso de numero de nucleos del cpu.**

In [19]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# Inicializar SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Definir la función de análisis de sentimiento
def analyze_sentiment(text):
    sentiment_scores = sia.polarity_scores(text)
    compound_score = sentiment_scores['compound']
    if compound_score < -0.05:
        return 0  # Malo
    elif compound_score > 0.05:
        return 2  # Positivo
    else:
        return 1  # Neutral

# Determinar el número de núcleos de CPU disponibles
num_cores = 5  # Reemplaza <Número_de_núcleos> con el número de núcleos que deseas utilizar

# Aplicar análisis de sentimiento en paralelo
sentiments = Parallel(n_jobs=num_cores)(delayed(analyze_sentiment)(text) for text in df_google_estados['text'])

# Agregar los resultados al DataFrame
df_google_estados['sentiment_analysis'] = sentiments

# Mostrar las primeras filas del DataFrame
df_google_estados.head()

StatementMeta(, 652a407c-4baa-4bc2-bae0-2be59ae1e377, 21, Finished, Available)

[nltk_data] Downloading package vader_lexicon to /home/trusted-
[nltk_data]     service-user/nltk_data...
/tmp/ipykernel_5644/3216807074.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_google_estados['sentiment_analysis'] = sentiments


,user_id,name,rating,text,gmap_id,estado,fecha,sentiment_analysis
0,1.140438e+20,Kanisha Mixon,5,Very Personable staff! Beautiful and clean env...,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama,2020-08-11,2
1,1.160090e+20,Brandie Hodges,5,Best clothing intown,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama,2021-01-06,2
2,1.062399e+20,Sharon King,4,S/D,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama,2019-01-11,1
3,1.049701e+20,Veronica Pierce,5,S/D,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama,2018-02-04,1
4,1.105875e+20,Whitney Waldon Collier,5,S/D,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf,Alabama,2018-08-26,1


In [20]:
# Reemplazar los valores float (posiblemente NaNs) por 'S/D' en la columna 'name'
df_google_estados['name'] = df_google_estados['name'].fillna('S/D')

# Mostrar información actualizada del DataFrame
print(df_google_estados.info())

StatementMeta(, 652a407c-4baa-4bc2-bae0-2be59ae1e377, 22, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
Index: 27619402 entries, 0 to 78201735
Data columns (total 8 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             float64
 1   name                object 
 2   rating              int64  
 3   text                object 
 4   gmap_id             object 
 5   estado              object 
 6   fecha               object 
 7   sentiment_analysis  int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 1.9+ GB
None


/tmp/ipykernel_5644/1401517714.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_google_estados['name'] = df_google_estados['name'].fillna('S/D')


In [21]:
#Cargamos la data a nuestro lakehouse
df_google_estados.to_csv('abfss://558991ad-6fdf-463a-b58a-238ddabaa398@onelake.dfs.fabric.microsoft.com/37a0382d-f783-4728-a697-48b1dc002937/Files/DATA FINAL/GOOGLE MAPS/Estados_filtrado.csv',escapechar='\\')

StatementMeta(, 652a407c-4baa-4bc2-bae0-2be59ae1e377, 23, Finished, Available)

###### **PARA LA DATA DE METADATA:**